# Prototyping the whole scenario

---



In [37]:
from pydantic import BaseModel, Field
from typing import List, Literal, Any, Optional,Tuple
from datetime import datetime


Verdict = Literal["A", "R", "E"]  # Answer, Request Info, Escalate


class SupportTicketState(BaseModel):
    # -------------------------
    # Identity
    # -------------------------
    ticket_id: str

    # -------------------------
    # User input
    # -------------------------
    ticket_text: str = Field(description="Latest user message")
    messages: List[Tuple[str,str]] = Field(default_factory=list)

    # -------------------------
    # Extracted information
    # -------------------------
    info_list: List[str] = Field(
        default_factory=list,
        description="All confirmed facts provided by the user"
    )

    # -------------------------
    # Knowledge & retrieval
    # -------------------------
    retrieved_from_kb: List[Any] = Field(
        default_factory=list,
        description="KB + past ticket retrieval results"
    )

    # -------------------------
    # Decision intelligence
    # -------------------------
    confidence: float = Field(
        ge=0.0,
        le=1.0,
        description="Confidence in ability to auto-resolve"
    )

    current_verdict: Optional[Verdict] = None
    final_verdict: Optional[Verdict] = None

    number_of_request_infos: int = 0

    # -------------------------
    # SLA & timing (NON-NEGOTIABLE)
    # -------------------------
    ticket_created_at: datetime
    last_agent_action_at: Optional[datetime] = None
    last_user_message_at: Optional[datetime] = None

    sla_seconds: int = Field(description="Maximum allowed response time")

    # -------------------------
    # Output (only valid if ANSWER)
    # -------------------------
    answer: Optional[str] = None


In [38]:
class ExtractedFormat(BaseModel):
    facts: List[str] = Field(
        description="Explicit factual statements stated by the user, not assumptions"
    )

In [39]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a fact extraction component for a technical support system.

Rules:
- Extract ONLY information explicitly stated by the user
- Do NOT infer or guess
- Do NOT troubleshoot or explain
- Do NOT include opinions or emotions
- If no facts are present, return an empty list

Return the result strictly in the required structured format.
"""),
    ("human", "{message}")
])


extract_llm = ChatGroq(model='llama-3.1-8b-instant').with_structured_output(ExtractedFormat)


extract_chain = prompt | extract_llm

In [40]:
extracted = extract_chain.invoke({"message": 'I use Arch btw'})
extracted


ExtractedFormat(facts=['Arch'])

In [41]:
from datetime import datetime

def parse_and_update_state(state, user_input):
    # Update raw input
    state.ticket_text = user_input
    state.messages.append(user_input)

    # Extract facts (structured output)
    extracted = extract_chain.invoke({"message": user_input})

    # Merge facts safely (idempotent behavior)
    for fact in extracted.facts:
        if fact not in state.info_list:
            state.info_list.append(fact)

    # Timestamp user interaction
    state.last_user_message_at = datetime.utcnow()

    return state


In [42]:
from datetime import datetime, timedelta

example_state = SupportTicketState(
    ticket_id="TICKET-123",
    ticket_text="",
    messages=[
        ('user',"My app crashes when I open it")
    ],
    info_list=[
        "App crashes on launch"
    ],
    retrieved_from_kb=[],
    confidence=0.0,
    current_verdict=None,
    final_verdict=None,
    number_of_request_infos=1,
    ticket_created_at=datetime.utcnow() - timedelta(hours=2),
    last_agent_action_at=datetime.utcnow() - timedelta(minutes=30),
    sla_seconds=24 * 60 * 60,  # 24 hours
    answer=None
)


C:\Users\DELL\AppData\Local\Temp\ipykernel_21892\3803734837.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ticket_created_at=datetime.utcnow() - timedelta(hours=2),
C:\Users\DELL\AppData\Local\Temp\ipykernel_21892\3803734837.py:18: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  last_agent_action_at=datetime.utcnow() - timedelta(minutes=30),


In [43]:
user_input = "I am using macOS Ventura on version 2.3.1"

parse_and_update_state(example_state, user_input=user_input)

C:\Users\DELL\AppData\Local\Temp\ipykernel_21892\1959626494.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  state.last_user_message_at = datetime.utcnow()


SupportTicketState(ticket_id='TICKET-123', ticket_text='I am using macOS Ventura on version 2.3.1', messages=[('user', 'My app crashes when I open it'), 'I am using macOS Ventura on version 2.3.1'], info_list=['App crashes on launch', 'macOS Ventura', 'version 2.3.1'], retrieved_from_kb=[], confidence=0.0, current_verdict=None, final_verdict=None, number_of_request_infos=1, ticket_created_at=datetime.datetime(2026, 1, 11, 1, 47, 55, 204589), last_agent_action_at=datetime.datetime(2026, 1, 11, 3, 17, 55, 204589), last_user_message_at=datetime.datetime(2026, 1, 11, 3, 47, 56, 436181), sla_seconds=86400, answer=None)